## EE 782: Advanced Topics in Machine Learning
### **Course Project** Person Re-Identification through SSL using DINO

### Collaborators: 
[![Munish](https://img.shields.io/badge/22M2153-Munish_Monga-blue)](https://github.com/munish30monga)
[![Uddeshya](https://img.shields.io/badge/22M2152-Uddeshya_Singh-darkgreen)](https://github.com/uddeshya-iitb)

> [Github Repo Link](https://github.com/munish30monga/person_ReID)

### Table of Contents:
- [Introduction](#introduction)
- [Importing Libraries](#importing-libraries)
- [Hyperparameters](#hyperparameters)


### Introduction <a id="introduction"></a>

### Importing Libraries <a id="importing-libraries"></a>